In [12]:
from multiprocess import Pool

file = open('data/19.txt', 'r')
data = file.read()

file.close()

# Random seach

This is my first ´random search´ solution that I did for fun but it worked so great that I took both stars with it and it gave best score of all days.

Scroll down for proper solution.

In [13]:
import re
import random

RESOURCES = (1, 0.8, 0.5, 0.5)

class Factory:
    def __init__(self, blueprint):
        # Geode, Obsidian, Clay, Ore
        self.resources = [ 0, 0, 0, 0 ]

        self.costs = [
            [ 0, blueprint[5], 0, blueprint[4] ],
            [ 0, 0, blueprint[3], blueprint[2] ],
            [ 0, 0, 0,            blueprint[1] ],
            [ 0, 0, 0,            blueprint[0] ]
        ]

    def can_build(self, robot):
        cost = self.costs[robot]

        for i in range(4):
            if cost[i] > self.resources[i]:
                return False

        return True

    def reduce_resources(self, robot):
        cost = self.costs[robot]

        for i in range(4):
            self.resources[i] -= cost[i]

    def build(self):
        if self.resources[-1] == 0:
            return None

        # Fuck algorithms, let's be creative. 
        # Simulate harvest and decide randomly to build robots or not!
        for n, p in enumerate(RESOURCES):
            if self.can_build(n) and random.random() < p:
                self.reduce_resources(n)
                return n

        return None

In [14]:
blueprints = []

for line in data.splitlines():
    _, *blueprint = map(int, re.findall(r'\d+', line))
    blueprints.append(blueprint)

In [15]:
def harvest(factory: Factory, time: int):
    robots = [3]

    for minute in range(time):
        new = None

        if minute > 0 and minute < time - 1:
            new = factory.build()

        for robot in robots:
            factory.resources[robot] += 1

        if new != None:
            robots.append(new)

    return factory.resources[0]

def process_blueprint(blueprint, cycles: int, time: int):
    compute = lambda _: harvest(Factory(blueprint), time)
    
    with Pool(processes=16) as pool:
        return max(pool.map(compute, range(cycles)))        

In [16]:
geodes = [process_blueprint(b, 10000, 24) for b in blueprints]
answer = 0

for i, v in enumerate(geodes):
    answer += v * (i + 1)

answer

1599

In [17]:
geodes = [process_blueprint(b, 1000000, 32) for b in blueprints[:3]]
answer = 1

for _, v in enumerate(geodes):
    answer *= v

answer

14112

# DFS search with optimizations

I keep state in a tuple of complex numbers where real part is amount of resources, imaginary part is amount of robots - just to keep code shorter.

Instead of building minute by minute, I calculate time needed to build each type of robot with given resources and fast-forward useless branches. I always build geode robot if possible and I don not build more robots than resources needed to build any robot which depends on it: we can build only one robot at a time anyway.

In [170]:
import re
from math import ceil

with open('data/19.txt', 'r') as file:
    data = file.read()

blueprints = []

for line in data.splitlines():
    _, *blueprint = map(int, re.findall(r'\d+', line))
    blueprints.append(blueprint)

def collect(state, times):
    return [complex(v.real + v.imag * times, v.imag) for v in state]

def pay(state, robot, n):
    paid = [a - b for a, b in zip(state, robot)]
    paid[n] += 1j
    
    return tuple(paid)

def build(state, costs, time, limit):
    for n, robot in enumerate(costs):
        # Do not build useless robots (we already have enough to fullfil any robot in 1 minute)
        if n > 0 and state[n].imag == max(map(lambda c: c[n], costs)):
            continue

        # Resources and costs for the robot
        resources = [(a, b) for a, b in zip(state, robot) if b]

        # If we have robots to harvest resource for this robot
        if all(map(lambda a: a[0].imag, resources)):
            # When the robot can be built?
            wait = max(ceil((b - a.real) / a.imag) for a, b in resources)
            
            # Probably, now, anyway add 1 minute to build it
            wait = 1 if wait < 0 else wait + 1

            if time + wait <= limit:
                collected = collect(state, wait)
                yield pay(collected, robot, n), wait

                # If we can build geode now, let's not consider other options
                if n == 0 and wait == 1:
                    break

In [171]:
def dfs(blueprint, limit):
    costs = [
        [0, blueprint[5], 0, blueprint[4]],
        [0, 0, blueprint[3], blueprint[2]],
        [0, 0, 0,            blueprint[1]],
        [0, 0, 0,            blueprint[0]],
    ]

    time = 0
    geodes = 0
    start = (0j, 0j, 0j, 1j)
    visited = set((start, time))
    queue = [(start, time)]

    while queue:
        state, time = queue.pop()

        if time == limit:
            geodes = max(geodes, int(state[0].real))
            continue

        for neighbour, steps in build(state, costs, time, limit):
            candidate = (neighbour, time + steps)

            if candidate not in visited:
                queue.append(candidate)
                visited.add(candidate)

    return geodes

# Star 1

In [173]:
answer = 0

for i, blueprint in enumerate(blueprints):
    answer += dfs(blueprint, 24) * (i + 1)

int(answer)

1599

# Star 2

In [169]:
answer = 1

for blueprint in blueprints[:3]:
    answer *= dfs(blueprint, 32)

int(answer)

14112